In [13]:
import pandas as pd
import numpy as np
import datetime as dt

In [1]:
!python -m pip install pandas --force-reinstall --user

  Using cached pandas-2.2.2-cp39-cp39-win_amd64.whl (11.6 MB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl (15.8 MB)
  Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2024.1
    Uninstalling tzdata-2024.1:
      Successfully uninstalled tzdata-2024.1
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.9.0.post0
    Uninstalling python-dateutil-2.9.0.post0:
      Successfully uninstalled python-dateutil-2.9.0.post0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully un

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\spejic1\\AppData\\Roaming\\Python\\Python39\\site-packages\\~%mpy.libs\\libopenblas64__v0.3.23-293-gc2f4bdbb-gcc_10_3_0-2bde3a66a51006b2b53eb373ff767a3f.dll'
Check the permissions.

You should consider upgrading via the 'c:\Users\spejic1\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [14]:
filename = 'CCDA_6771_Extract_03042024.xlsx'
sheet_name = 'Mar_Data'
df = pd.read_excel(filename, sheet_name=sheet_name)

In [15]:
df = df.drop(['MRN','CSN', 'pat_id', 'pat_Newid','Enc_NewID','order_med_id','dose','generic_name','description'], axis=1)

In [16]:
df.head()

,Last_Name,Date_Time_start,Date_Time_end,mar_time,mar_action,med_name
0,Wojeski,2023-11-03 16:15:00,2023-11-04 16:45:00,2023-10-18 13:12:00,Missed,PHYTONADIONE (VITAMIN K1) 1 MG/0.5 ML INJECTIO...
1,Wojeski,2023-11-03 16:15:00,2023-11-04 16:45:00,2023-11-03 16:29:00,Stopped,DEXMEDETOMIDINE INFUSION 12 MCG/ML IN 100 ML (...
2,Wojeski,2023-11-03 16:15:00,2023-11-04 16:45:00,2023-10-18 07:30:00,Due,FENTANYL 4 MCG/ML IV DILUTION
3,Wojeski,2023-11-03 16:15:00,2023-11-04 16:45:00,2023-10-18 10:20:00,Given,PHYTONADIONE (VITAMIN K1) 1 MG/0.5 ML INJECTIO...
4,Wojeski,2023-11-03 16:15:00,2023-11-04 16:45:00,2023-10-18 08:00:00,Due,ROCURONIUM 10 MG/ML INTRAVENOUS SOLUTION


In [17]:
df['mar_time'] = pd.to_datetime(df['mar_time'])
df['Date_Time_start'] = pd.to_datetime(df['Date_Time_start'])
df['Date_Time_end'] = pd.to_datetime(df['Date_Time_end'])
df = df[(df['mar_time'] >= df['Date_Time_start']) & (df['mar_time'] <= df['Date_Time_end'])]
df = df.sort_values(by=['Last_Name','mar_time'])
df = df.drop(['Date_Time_start','Date_Time_end'], axis=1)
df.head()

,Last_Name,mar_time,mar_action,med_name
31649,Burgos,2023-12-12 16:00:00,Rate Verify,DEXMEDETOMIDINE INFUSION 12 MCG/ML IN 100 ML (...
31908,Burgos,2023-12-12 16:00:00,Rate Verify,DEXTROSE 5 % AND LACTATED RINGERS INTRAVENOUS ...
31971,Burgos,2023-12-12 16:14:00,Given,DIAZEPAM 5 MG/5 ML (1 MG/ML) ORAL SOLUTION
31710,Burgos,2023-12-12 17:00:00,Rate Verify,DEXMEDETOMIDINE INFUSION 12 MCG/ML IN 100 ML (...
31779,Burgos,2023-12-12 17:00:00,Rate Verify,DEXTROSE 5 % AND LACTATED RINGERS INTRAVENOUS ...


In [18]:
sedatives = ["DEXMEDETOMIDINE", "FENTANYL", "MIDAZOLAM","PROPOFOL","HYDROMORPHONE", "DIAZEPAM","LORAZEPAM", "KETAMINE"]
blockers = ["ROCURONIUM","VECURONIUM","CISATRACURIUM"]
# turn all of med_name column uppercase
df['med_name'] = df['med_name'].str.upper()

### CODE TO FIND ALL NEUROMUSCULAR BLOCKER DATA ###

In [19]:
data_blockers = df[df['med_name'].apply(lambda x: any([drug in x for drug in blockers]))]
data_blockers = data_blockers[~data_blockers['mar_action'].isin(['Missed','Stopped', 'Canceled Entry', 'Return to Cabinet', 'Override Pull' 'Due'])]
count = data_blockers['Last_Name'].value_counts()
print(count)
data_blockers[220:280]

Last_Name
Pediford    209
Faurot       12
Wojeski       7
Seo           1
Name: count, dtype: int64


,Last_Name,mar_time,mar_action,med_name
42594,Pediford,2024-01-30 11:00:00,Rate Verify,VECURONIUM INFUSION 1 MG/ML (10 MG VIAL) (PEDS)
46827,Seo,2024-01-22 16:04:00,Due,ROCURONIUM 10 MG/ML INTRAVENOUS SOLUTION
5491,Wojeski,2023-11-04 08:40:00,See Alternative,VECURONIUM BOLUS FROM BAG (JHH PEDS)
5249,Wojeski,2023-11-04 08:41:00,See Alternative,VECURONIUM BOLUS FROM BAG (JHH PEDS)
6349,Wojeski,2023-11-04 08:41:00,Given,VECURONIUM BROMIDE 10 MG INTRAVENOUS SOLUTION
5273,Wojeski,2023-11-04 16:30:00,Given,VECURONIUM BROMIDE 10 MG INTRAVENOUS SOLUTION
6007,Wojeski,2023-11-04 16:30:00,See Alternative,VECURONIUM BOLUS FROM BAG (JHH PEDS)
5257,Wojeski,2023-11-04 16:45:00,See Alternative,VECURONIUM BOLUS FROM BAG (JHH PEDS)
5692,Wojeski,2023-11-04 16:45:00,Given,VECURONIUM BROMIDE 10 MG INTRAVENOUS SOLUTION


In [20]:
# get only start/stop times from entire dataframe

data_blockers_fin = pd.DataFrame(columns=data_blockers.columns)
data_blockers_fin.head()

patients = np.unique(data_blockers['Last_Name'])
for name in patients:
    patient_data = data_blockers[data_blockers['Last_Name'] == name]
    patient_data_fin = pd.DataFrame(columns=data_blockers.columns)
    patient_data_fin = pd.concat([patient_data_fin,patient_data.iloc[[0]]], ignore_index=True)
    
    for i in range(1, patient_data.shape[0] - 1):
        time_diff_with_previous = (patient_data.iloc[i]['mar_time']-patient_data.iloc[i-1]['mar_time']).total_seconds()
        time_diff_with_next = (patient_data.iloc[i+1]['mar_time']-patient_data.iloc[i]['mar_time']).total_seconds()
        if time_diff_with_next > (3*3600):
            # end_time = patient_data.iloc[i]['mar_time']
            patient_data_fin = pd.concat([patient_data_fin, patient_data.iloc[[i]]], ignore_index=True)
        if time_diff_with_previous > (3*3600):
            # start_time = patient_data.iloc[i]['mar_time']
            patient_data_fin = pd.concat([patient_data_fin, patient_data.iloc[[i]]], ignore_index=True)
        if i == patient_data.shape[0] - 2:
            patient_data_fin = pd.concat([patient_data_fin, patient_data.iloc[[-1]]], ignore_index=True)
            

    data_blockers_fin = pd.concat([data_blockers_fin,patient_data_fin], ignore_index=True)
    
data_blockers_fin.head(30)  

C:\Users\spejic1\AppData\Local\Temp\ipykernel_12152\2812282024.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  patient_data_fin = pd.concat([patient_data_fin,patient_data.iloc[[0]]], ignore_index=True)
C:\Users\spejic1\AppData\Local\Temp\ipykernel_12152\2812282024.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_blockers_fin = pd.concat([data_blockers_fin,patient_data_fin], ignore_index=True)
C:\Users\spejic1\AppData\Local\Temp\ipykernel_12152\2812282024.py:10: FutureWarning: The b

,Last_Name,mar_time,mar_action,med_name
0,Faurot,2023-11-14 10:45:00,New Bag,VECURONIUM INFUSION 1 MG/ML (10 MG VIAL) (PEDS)
1,Faurot,2023-11-14 12:00:00,Rate Verify,VECURONIUM INFUSION 1 MG/ML (10 MG VIAL) (PEDS)
2,Faurot,2023-11-16 10:01:00,New Bag,VECURONIUM INFUSION 1 MG/ML (10 MG VIAL) (PEDS)
3,Faurot,2023-11-16 15:00:00,Rate Verify,VECURONIUM INFUSION 1 MG/ML (10 MG VIAL) (PEDS)
4,Pediford,2024-01-22 15:00:00,Rate Verify,VECURONIUM INFUSION 1 MG/ML (10 MG VIAL) (PEDS)
5,Pediford,2024-01-22 22:00:00,Rate Verify,VECURONIUM INFUSION 1 MG/ML (10 MG VIAL) (PEDS)
6,Pediford,2024-01-23 13:40:00,Given,VECURONIUM BROMIDE 10 MG INTRAVENOUS SOLUTION
7,Pediford,2024-01-30 11:00:00,Rate Verify,VECURONIUM INFUSION 1 MG/ML (10 MG VIAL) (PEDS)
8,Seo,2024-01-22 16:04:00,Due,ROCURONIUM 10 MG/ML INTRAVENOUS SOLUTION
9,Wojeski,2023-11-04 08:40:00,See Alternative,VECURONIUM BOLUS FROM BAG (JHH PEDS)


### CODE TO FIND ALL SEDATIVES DATA ###

In [21]:
data_sedatives = df[df['med_name'].apply(lambda x: any([drug in x for drug in sedatives]))]
# exclude any orally administered and PCA IV and IV PCA for pain not sedation
data_sedatives = data_sedatives[~data_sedatives['med_name'].str.contains('ORAL|INJECTION|BOLUS')]
data_sedatives = data_sedatives[~(data_sedatives['med_name'].str.contains('PCA') & ~data_sedatives['med_name'].str.contains('KETAMINE'))]
# remove all entries not given
data_sedatives = data_sedatives[~data_sedatives['mar_action'].isin(['Missed','Stopped', 'Canceled Entry', 'Return to Cabinet', 'Override Pull', 'Due'])]
# count how many entries per patient
count = data_sedatives['Last_Name'].value_counts()
print(count)
# data_sedatives[450:475]

Last_Name
Pediford    780
Song        575
Faurot      440
Seo         296
Nusbaum     114
Smith        75
Lebovits     68
Wojeski      47
Reed         45
Mack         38
Burgos       30
Name: count, dtype: int64


In [22]:
# get only start/stop times from entire dataframe

data_sedatives_fin = pd.DataFrame(columns=data_sedatives.columns)
data_sedatives_fin.head()

patients = np.unique(data_sedatives['Last_Name'])
for name in patients:
    patient_data = data_sedatives[data_sedatives['Last_Name'] == name]
    patient_data_fin = pd.DataFrame(columns=data_sedatives.columns)
    patient_data_fin = pd.concat([patient_data_fin,patient_data.iloc[[0]]], ignore_index=True)
    
    for i in range(1, patient_data.shape[0] - 1):
        time_diff_with_previous = (patient_data.iloc[i]['mar_time']-patient_data.iloc[i-1]['mar_time']).total_seconds()
        time_diff_with_next = (patient_data.iloc[i+1]['mar_time']-patient_data.iloc[i]['mar_time']).total_seconds()
        if time_diff_with_next > (3*3600):
            # end_time = patient_data.iloc[i]['mar_time']
            patient_data_fin = pd.concat([patient_data_fin, patient_data.iloc[[i]]], ignore_index=True)
        if time_diff_with_previous > (3*3600):
            # start_time = patient_data.iloc[i]['mar_time']
            patient_data_fin = pd.concat([patient_data_fin, patient_data.iloc[[i]]], ignore_index=True)
        if i == patient_data.shape[0] - 2:
            patient_data_fin = pd.concat([patient_data_fin, patient_data.iloc[[-1]]], ignore_index=True)
            

    data_sedatives_fin = pd.concat([data_sedatives_fin,patient_data_fin], ignore_index=True)
    
data_sedatives_fin.head(30)  

C:\Users\spejic1\AppData\Local\Temp\ipykernel_12152\340974634.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  patient_data_fin = pd.concat([patient_data_fin,patient_data.iloc[[0]]], ignore_index=True)
C:\Users\spejic1\AppData\Local\Temp\ipykernel_12152\340974634.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_sedatives_fin = pd.concat([data_sedatives_fin,patient_data_fin], ignore_index=True)
C:\Users\spejic1\AppData\Local\Temp\ipykernel_12152\340974634.py:10: FutureWarning: The be

,Last_Name,mar_time,mar_action,med_name
0,Burgos,2023-12-12 16:00:00,Rate Verify,DEXMEDETOMIDINE INFUSION 12 MCG/ML IN 100 ML (...
1,Burgos,2023-12-13 16:00:00,Rate Verify,DEXMEDETOMIDINE INFUSION 12 MCG/ML IN 100 ML (...
2,Faurot,2023-11-13 18:00:00,Rate Verify,FENTANYL INFUSION 10 MCG/ML IN 50 ML (JHH ACH ...
3,Faurot,2023-11-24 01:00:00,Rate Verify,DEXMEDETOMIDINE INFUSION 4 MCG/ML (JHH PED)
4,Lebovits,2024-02-15 16:00:00,Rate Verify,DEXMEDETOMIDINE INFUSION 12 MCG/ML IN 100 ML (...
5,Lebovits,2024-02-18 03:00:00,Rate Verify,DEXMEDETOMIDINE INFUSION 12 MCG/ML IN 100 ML (...
6,Mack,2024-02-02 09:00:00,Rate Verify,DEXMEDETOMIDINE INFUSION 4 MCG/ML (JHH PED)
7,Mack,2024-02-03 15:00:00,Rate Verify,DEXMEDETOMIDINE INFUSION 4 MCG/ML (JHH PED)
8,Nusbaum,2023-10-27 12:00:00,Rate Verify,DEXMEDETOMIDINE INFUSION 2 MCG/ML IN 50 ML (JH...
9,Nusbaum,2023-10-31 09:00:00,Rate Verify,DEXMEDETOMIDINE INFUSION 2 MCG/ML IN 50 ML (JH...


### CODE TO FIND ALL SEDATIVE PRN TIMES ###

In [23]:
data_prn = df[df['med_name'].apply(lambda x: any([drug in x for drug in sedatives]))]
data_prn = data_prn[~data_prn['mar_action'].isin(['Missed','Stopped', 'Canceled Entry', 'Return to Cabinet', 'Override Pull', 'Due', 'See Alternative'])]
data_prn = data_prn[~data_prn['med_name'].str.contains('INFUSION')]
count = data_prn['Last_Name'].value_counts()
print(count)
# data_prn.to_csv('prn_admin_04082024.csv')
data_prn.head()

Last_Name
Pediford    177
Seo         119
Faurot      100
Song         60
Nusbaum      35
Smith        25
Wojeski      20
Mack         14
Lebovits      9
Burgos        7
Name: count, dtype: int64


,Last_Name,mar_time,mar_action,med_name
31971,Burgos,2023-12-12 16:14:00,Given,DIAZEPAM 5 MG/5 ML (1 MG/ML) ORAL SOLUTION
31964,Burgos,2023-12-12 22:33:00,Given,DIAZEPAM 5 MG/5 ML (1 MG/ML) ORAL SOLUTION
32024,Burgos,2023-12-13 04:16:00,Given,DIAZEPAM 5 MG/5 ML (1 MG/ML) ORAL SOLUTION
32035,Burgos,2023-12-13 10:11:00,Given,DIAZEPAM 5 MG/5 ML (1 MG/ML) ORAL SOLUTION
32011,Burgos,2023-12-13 15:53:00,Given,DIAZEPAM 5 MG/5 ML (1 MG/ML) ORAL SOLUTION


### CODE TO SET UP SCORING SPREADSHEET ###

In [24]:
# NOTE: this code will have to be changed with each patient last name
patient = 'Mack'

# create sbs_df with default 30 min increments during enrollment
sbs_df = pd.DataFrame(columns=['Datetime','SBS','Stim?','Stim-Y-type?','Default?','SedPRN?','SedPRN-Y-type?','BlockPRN?','Notes'])
linking_df = pd.read_excel('linking table.xlsx')
start_time = linking_df.loc[linking_df['Last Name'] == patient, 'Date-Time start']
start_time = pd.to_datetime(start_time.iloc[0])
end_time = linking_df.loc[linking_df['Last Name'] == patient, 'Date-Time end']
end_time = pd.to_datetime(end_time.iloc[0])
time_range = pd.date_range(start=start_time, end=end_time, freq='30min')
time_df = pd.DataFrame({'Datetime': time_range},columns=['Datetime'])
sbs_df['Datetime']=time_df['Datetime']

# insert rows according to datetimes of sedative PRN administration
patient_prn = data_prn.loc[data_prn['Last_Name']==patient]
patient_prn['mar_time'] = pd.to_datetime(patient_prn['mar_time'])
merged_time_df = pd.concat([sbs_df['Datetime'], patient_prn['mar_time']], ignore_index=True)
merged_time_df.sort_values(inplace=True)
merged_time_df.reset_index(drop=True,inplace=True)
sbs_prn_df = pd.DataFrame(columns=['Datetime','SBS','Stimulation?','Stim-Y-type?','Default?','SedPRN?','SedPRN-Y-type?','BlockPRN?','Notes'])
sbs_prn_df['Datetime']=merged_time_df
sbs_prn_df['SedPRN?']='N'
sbs_prn_df['BlockPRN?']='N'
sbs_prn_df['SedPRN-Y-type?']=''
sbs_prn_df['Lights on?'] = 'Y'
sbs_prn_df['In bed?'] = 'Y'
sbs_prn_df['StimDur'] = ''
for index,datetime in sbs_prn_df['Datetime'].items():
    if datetime in patient_prn['mar_time'].tolist():
        sbs_prn_df.at[index,'SedPRN?']='Y'
        matching_row = patient_prn[patient_prn['mar_time']==datetime]
        med_name = matching_row['med_name'].str.split().str[0].values[0]
        sbs_prn_df.at[index,'SedPRN-Y-type?'] = med_name

for ind,datetime in sbs_prn_df['Datetime'].items():
    if datetime in patient_prn['mar_time'].tolist():
        if ind+1 < len(sbs_prn_df):
            patient_prn.reset_index(drop=True, inplace=True)
            prn_df_ind = patient_prn.index[patient_prn['mar_time'] == datetime]
            if prn_df_ind[0]+1 < len(patient_prn):
                next_mar_time = patient_prn.iloc[prn_df_ind+1]['mar_time'].values
                if next_mar_time[0]>=datetime+dt.timedelta(minutes=30):
                    sbs_prn_df.at[ind+1,'Datetime'] = datetime+dt.timedelta(minutes=30)
                    sbs_prn_df.at[ind+1,'SedPRN?']='Y' 

# insert rows according to datetimes of blocker PRN administration
patient_nb_prn = data_blockers_fin.loc[data_blockers_fin['Last_Name']==patient] 
if not patient_nb_prn.empty:
    print('entered neuromuscular blockers loop')
    patient_nb_prn['mar_time'] = pd.to_datetime(patient_nb_prn['mar_time'])
    patient_nb_prn.head()

    for index,row in patient_nb_prn.iterrows():
        insert_time = row['mar_time']
        insert_index = sbs_prn_df.index[sbs_prn_df['Datetime']>=insert_time].min()
        sbs_prn_df = pd.concat([sbs_prn_df.iloc[:insert_index],
                                pd.DataFrame({'Datetime':[insert_time],'BlockPRN?':['Y']}),
                                sbs_prn_df.iloc[insert_index:]]).reset_index(drop=True)
        insert_1h = sbs_prn_df.index[sbs_prn_df['Datetime']>=insert_time+dt.timedelta(hours=1)].min()
        if not np.isnan(insert_1h):
            sbs_prn_df = pd.concat([sbs_prn_df.iloc[:insert_1h],
                                    pd.DataFrame({'Datetime':[insert_time+dt.timedelta(hours=1)],'BlockPRN?':['Y']}),
                                    sbs_prn_df.iloc[insert_1h:]]).reset_index(drop=True)
            sbs_prn_df.loc[(sbs_prn_df['Datetime']>insert_time) & (sbs_prn_df['Datetime']<insert_time+dt.timedelta(hours=1)), 'BlockPRN?'] = 'Y'
        
sbs_prn_df.to_csv('sbs_p12_bp.csv')

C:\Users\spejic1\AppData\Local\Temp\ipykernel_12152\623659791.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_prn['mar_time'] = pd.to_datetime(patient_prn['mar_time'])
